# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
from tavily import TavilyClient

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import Tool
from lib.evaluation import EvaluationReport
from lib.parsers import PydanticOutputParser

In [3]:
# Load environment variables and Vocareum OpenAI config
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
VOCAREUM_BASE_URL = "https://openai.vocareum.com/v1"

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [4]:
# Chroma client and collection (same path and embedding as Part 1)
_chroma_client = chromadb.PersistentClient(path="chromadb")
_embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key_env_var="OPENAI_API_KEY",
    api_base=VOCAREUM_BASE_URL,
    model_name="text-embedding-3-small",
)
_game_collection = _chroma_client.get_collection(name="udaplay", embedding_function=_embedding_fn)


def retrieve_game(query: str) -> str:
    """Semantic search: Finds the most relevant results in the vector DB.
    Args:
        query: A question about the game industry.
    Returns:
        A list of results. Each element contains:
        - Platform: e.g. Game Boy, PlayStation 5, Xbox 360
        - Name: Name of the game
        - YearOfRelease: Year when that game was released for that platform
        - Description: Additional details about the game
    """
    n_results = 5
    results = _game_collection.query(query_texts=[query], n_results=n_results, include=["metadatas", "documents"])
    if not results or not results["metadatas"] or not results["metadatas"][0]:
        return json.dumps([])
    items = []
    for meta, doc in zip(results["metadatas"][0], results["documents"][0]):
        items.append({
            "Platform": meta.get("Platform", ""),
            "Name": meta.get("Name", ""),
            "YearOfRelease": meta.get("YearOfRelease"),
            "Description": meta.get("Description", doc or ""),
        })
    return json.dumps(items, default=str)


retrieve_game_tool = Tool(retrieve_game)

#### Evaluate Retrieval Tool

In [5]:
def evaluate_retrieval(question: str, retrieved_docs: str) -> str:
    """Evaluates whether the retrieved documents are sufficient to answer the question.
    Based on the user's question and the list of retrieved documents, analyzes
    the usability of the documents to respond to that question.
    Args:
        question: Original question from the user.
        retrieved_docs: JSON string of retrieved documents (list of dicts with Platform, Name, YearOfRelease, Description).
    Returns:
        A JSON object with: useful (bool), description (str) explaining the evaluation.
    """
    try:
        docs = json.loads(retrieved_docs) if isinstance(retrieved_docs, str) else retrieved_docs
    except (json.JSONDecodeError, TypeError):
        docs = []
    llm_judge = LLM(
        model="gpt-4o-mini",
        temperature=0.0,
        api_key=OPENAI_API_KEY,
        base_url=VOCAREUM_BASE_URL,
    )
    prompt = (
        "Your task is to evaluate if the documents are enough to respond to the query. "
        "Give a detailed explanation so it is possible to take an action to accept the retrieval or not.\n\n"
        f"Question: {question}\n\n"
        f"Retrieved documents:\n{json.dumps(docs, default=str, indent=2)}"
    )
    try:
        response = llm_judge.invoke(prompt, response_format=EvaluationReport)
        parser = PydanticOutputParser(model_class=EvaluationReport)
        report = parser.parse(response)
        return json.dumps({"useful": report.useful, "description": report.description})
    except Exception as e:
        return json.dumps({"useful": False, "description": f"Evaluation failed: {e}"})


evaluate_retrieval_tool = Tool(evaluate_retrieval)

#### Game Web Search Tool

In [6]:
def game_web_search(question: str) -> str:
    """Performs a web search for additional information about the game industry.
    Use when internal knowledge is insufficient to answer the question.
    Args:
        question: A question about the game industry (e.g. release dates, platforms, publishers).
    Returns:
        A string with relevant search results (snippets and sources).
    """
    if not TAVILY_API_KEY:
        return json.dumps({"error": "TAVILY_API_KEY not set", "results": []})
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question, search_depth="basic", max_results=5)
    results = response.get("results", [])
    snippets = [{"title": r.get("title", ""), "content": r.get("content", ""), "url": r.get("url", "")} for r in results]
    return json.dumps(snippets, default=str)


game_web_search_tool = Tool(game_web_search) 

### Agent

In [7]:
AGENT_INSTRUCTIONS = """You are UdaPlay, an AI research agent for the video game industry.

Your workflow:
1. First, use retrieve_game to search the internal vector database for relevant game information.
2. Use evaluate_retrieval to assess whether the retrieved documents are sufficient to answer the user's question.
3. If the retrieval is useful, answer from the retrieved documents. Cite Platform, Name, YearOfRelease, and Description when relevant.
4. If the retrieval is not useful or insufficient, use game_web_search to find information on the web, then answer from both sources.
5. Provide clear, structured answers. When citing, mention the source (e.g. "According to our database..." or "Web search indicates...").
6. If you cannot find an answer, say so clearly."""

agent = Agent(
    model_name="gpt-4o-mini",
    instructions=AGENT_INSTRUCTIONS,
    tools=[retrieve_game_tool, evaluate_retrieval_tool, game_web_search_tool],
    temperature=0.3,
    openai_api_key=OPENAI_API_KEY,
    openai_base_url=VOCAREUM_BASE_URL,
)

In [8]:
def run_query(agent: Agent, question: str, session_id: str = "default") -> None:
    """Run the agent on a question and print reasoning, tools used, and final answer (rubric)."""
    agent.reset_session(session_id)  # Fresh session per query so output clearly shows this query only
    print(f"\n{'='*60}\nQuery: {question}\n{'='*60}")
    run = agent.invoke(question, session_id=session_id)
    final_state = run.get_final_state()
    if not final_state:
        print("No final state.")
        return
    messages = final_state.get("messages", [])
    final_answer = None
    for msg in messages:
        if isinstance(msg, AIMessage):
            if msg.tool_calls:
                for tc in msg.tool_calls:
                    print(f"  [Tool] {tc.function.name}({tc.function.arguments})")
            if msg.content:
                final_answer = msg.content
        elif isinstance(msg, ToolMessage):
            content = msg.content[:200] + "..." if len(msg.content) > 200 else msg.content
            print(f"  [Tool result] {content}")
    if final_answer:
        print(f"Final answer (with citation if any): {final_answer}")
    print()

# Run agent on at least three example queries (per rubric)
example_queries = [
    "When was Pokémon Gold and Silver released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for PlayStation 5?",
]

for q in example_queries:
    run_query(agent, q)


Query: When was Pokémon Gold and Silver released?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
  [Tool] retrieve_game({"query":"Pokémon Gold and Silver release date"})
  [Tool result] "[{\"Platform\": \"Game Boy Color\", \"Name\": \"Pok\\u00e9mon Gold and Silver\", \"YearOfRelease\": 1999, \"Description\": \"Second-generation Pok\\u00e9mon games introducing new regions, Pok\\u00e9m...
  [Tool] evaluate_retrieval({"question":"When was Pokémon Gold and Silver released?","retrieved_docs":"[{\"Platform\": \"Game Boy Color\", \"Name\": \"Pok\\u00e9mon Gold and Silver\", \"YearOfRelease\": 1999, \"Description\": \"Second-generation Pok\\u00e9mon games introducing new regions, Pok\\u00e9mon, an

### (Optional) Advanced

In [9]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes